## Import

In [2]:
!jupyter nbconvert --to markdown /content/drive/MyDrive/Legend13/Self_Implement/psm_Inception_v1.ipynb

[NbConvertApp] Converting notebook /content/drive/MyDrive/Legend13/Self_Implement/psm_Inception_v1.ipynb to markdown
[NbConvertApp] Writing 18784 bytes to /content/drive/MyDrive/Legend13/Self_Implement/psm_Inception_v1.md


In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append("/content/drive/MyDrive/Legend13")
import torch 
import torch.nn as nn
!pip install torchinfo
from torchinfo import summary

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Inception V1 Architecture

<p align="center"><img src="https://github.com/SuminizZ/Physics/assets/92680829/86b9a02c-93b7-4c52-a71f-465a3e99a431" width="670px"><p>

<br/>

<p align="center"><img width="600px" alt="image" src="https://github.com/SuminizZ/Physics/assets/92680829/b9c4f89d-e0e6-4d2b-819a-5725cb549c51"></p>


1. improves computational efficiency 
    - reduce dimension of parameters by adding extra 1x1 conv layer before 3x3 and 5x5 conv layers 
    - take global average pooling before entering into fc layer 

2.  Use different sizes of filters to perform convolution on a single input and concatenate them into one output (output size adjusted with padding)
    - combination
    - recorded lowest error at ImageNet classifcation

3. mitigate gradient vanishing problem with auxiliary classifiers
    - increase the gradient signal from intermediate or lower layers but with regularization factor 0.3 (loss = final_out_loss + 0.3*(aux1_loss + aux2_loss)
    -> turns out this is not the case. (these branches not help reflecting low-level features) Instead, they work as regularizers with batch noramlization applied


<img src="https://github.com/SuminizZ/Physics/assets/92680829/29509567-5368-41f9-b847-538a95251520">

In [ ]:
class BasicConv2d(nn.Module):
    def __init__(self, in_channel, F, size, stride, padding=0):
        super().__init__()
        self.layer = nn.Sequential(nn.Conv2d(in_channel, F, size, stride, padding=padding),
                                   nn.BatchNorm2d(F),
                                   nn.ReLU())

    def forward(self, x):
        out = self.layer(x)
        return out 


class Inception(nn.Module):
    def __init__(self, in_channel, Fs, final_F):
        super().__init__()
        self.final_F = final_F
        c1_F, c3_red_F, c3_F, c5_red_F, c5_F, poolproj_F = Fs
        self.conv1x1 = BasicConv2d(in_channel, c1_F, 1, 1, padding=0)
        self.conv3x3 = nn.Sequential(BasicConv2d(in_channel, c3_red_F, 1, 1, padding=0),
                                     BasicConv2d(c3_red_F, c3_F, 3, 1, padding=1))
        self.conv5x5 = nn.Sequential(BasicConv2d(in_channel, c5_red_F, 1, 1, padding=0),
                                     BasicConv2d(c5_red_F, c5_F, 5, 1, padding=2))
        self.maxpool_conv1x1 = nn.Sequential(nn.MaxPool2d(3, 1, padding=1),
                                             BasicConv2d(in_channel, poolproj_F, 1, 1, padding=0))

    def forward(self, x):
        x_concat = [self.conv1x1(x), self.conv3x3(x), self.conv5x5(x), self.maxpool_conv1x1(x)]
        x_concat = torch.cat(x_concat, 1)   # N,F,H,W
        # assert x_concat.shape[1] == self.final_F
        
        return x_concat


class AuxOut(nn.Module):
    def __init__(self, in_channel, p, num_classes):
        super().__init__()
        self.avgpool_conv = nn.Sequential(nn.AvgPool2d(5, 3, padding=0),
                                          BasicConv2d(in_channel, 128, 1, 1, padding=0))
        self.fc = nn.Sequential(nn.Linear(2048, 1024),
                                nn.ReLU(),
                                nn.Dropout(p=p),
                                nn.Linear(1024, num_classes))
        
    def forward(self, x):
        x = self.avgpool_conv(x)
        x = torch.flatten(x,1)
        out = self.fc(x)
        return out 


class Inception_V1(nn.Module):
    def __init__(self, init_weights=True, p=0.5, use_aux=True, in_channel=3, num_classes=1000):
        super().__init__()
        self.use_aux = use_aux

        self.conv1 = BasicConv2d(in_channel, 64, 7, 2, padding=3)
        self.maxpool1 = nn.MaxPool2d(3, 2, padding=1)

        self.conv2a = BasicConv2d(64, 64, 1, 1, padding=0)
        self.conv2b = BasicConv2d(64, 192, 3, 1, padding=1)
        self.maxpool2 = nn.MaxPool2d(3, 2, padding=1)

        Fs = (64, 96, 128, 16, 32, 32)
        self.inception3a = Inception(192, Fs, 256)
        Fs = (128, 128, 192, 32, 96, 64)
        self.inception3b = Inception(256, Fs, 480)
        self.maxpool3 = nn.MaxPool2d(3, 2, padding=1)

        Fs = (192, 96, 208, 16, 48, 64)
        self.inception4a = Inception(480, Fs, 512)
        Fs = (160, 112, 224, 24, 64, 64)
        self.inception4b = Inception(512, Fs, 512)
        Fs = (128, 128, 256, 24, 64, 64)
        self.inception4c = Inception(512, Fs, 512)
        Fs = (112, 144, 288, 32, 64, 64)
        self.inception4d = Inception(512, Fs, 528)
        Fs = (256, 160, 320, 32, 128, 128)
        self.inception4e = Inception(528, Fs, 832)
        self.maxpool4 = nn.MaxPool2d(3, 2, padding=1)

        Fs = (256, 160, 320, 32, 128, 128)
        self.inception5a = Inception(832, Fs, 832)
        Fs = (384, 192, 384, 48, 128, 128)
        self.inception5b = Inception(832, Fs, 1024)

        if use_aux:
            self.aux1 = AuxOut(512, p, num_classes)
            self.aux2 = AuxOut(528, p, num_classes)
        else:
            self.aux1, self.aux2 = None, None
        
        self.gap = nn.AdaptiveAvgPool2d((1,1))
        self.dropout = nn.Dropout(p=p)
        self.classifier = nn.Linear(1024, num_classes)

        if init_weights:
            for m in self.modules():
                if isinstance(m, nn.Conv2d) or isinstance(m, nn.Conv2d):
                    nn.init.trunc_normal_(m.weight, 0, 1e-2, a=-2, b=2)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.BatchNorm2d):
                    nn.init.constant_(m.weight, 1)
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2a(x)
        x = self.conv2b(x)
        x = self.maxpool2(x)
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        x = self.inception4a(x)
        if self.training and self.aux1 is not None:
            aux1_out = self.aux1(x)
        else: aux1_out = None

        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)

        if self.training and self.aux2 is not None:
            aux2_out = self.aux2(x)
        else: aux2_out = None

        x = self.inception4e(x)
        x = self.maxpool4(x)
        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.gap(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        out = self.classifier(x)

        return aux1_out, aux2_out, out

## Summary

In [ ]:
model = Inception_V1()
summary(model, input_size=(2, 3, 224, 224), device='cpu')

Layer (type:depth-idx)                        Output Shape              Param #
Inception_V1                                  --                        6,380,240
├─BasicConv2d: 1-1                            [2, 64, 112, 112]         --
│    └─Sequential: 2-1                        [2, 64, 112, 112]         --
│    │    └─Conv2d: 3-1                       [2, 64, 112, 112]         9,472
│    │    └─BatchNorm2d: 3-2                  [2, 64, 112, 112]         128
│    │    └─ReLU: 3-3                         [2, 64, 112, 112]         --
├─MaxPool2d: 1-2                              [2, 64, 56, 56]           --
├─BasicConv2d: 1-3                            [2, 64, 56, 56]           --
│    └─Sequential: 2-2                        [2, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                       [2, 64, 56, 56]           4,160
│    │    └─BatchNorm2d: 3-5                  [2, 64, 56, 56]           128
│    │    └─ReLU: 3-6                         [2, 64, 56, 56]           --
├─Bas

## Forward Pass

In [ ]:
x = torch.randn(2,3,224,224)
aux1_out, aux2_out, out = model(x)
print(aux1_out.shape)
print(aux2_out.shape)
print(out.shape)

torch.Size([2, 1000])
torch.Size([2, 1000])
torch.Size([2, 1000])
